# Agata Załęska - Project assignment

## Preliminary analysis and cleaning

### Read data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.options.display.max_columns = None

path = './'
data = pd.read_csv(f"{path}/messy_data.csv")

print(f"Data columns are:\n {data.columns}\n")
print(f"{data.head(3)} \n")
print(f"Data size: {len(data)}")

### Adjust column names

In [ ]:
# delete leading and trailing spaces from columns names
data.columns = data.columns.str.strip()

print(f"Adjusted data columns are:\n {data.columns}\n")

### Take care of duplicates and missing data

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# deleting duplicates
data = data.drop_duplicates()

# check if data is consistent
# we see that it is not - only carat has always float value
print(data.dtypes)

#### Make the data consistent

In [ ]:
# convert other columns to correct types
data['price'] = pd.to_numeric(data['price'], errors='coerce')
data['carat'] = pd.to_numeric(data['carat'], errors='coerce')
data['x dimension'] = pd.to_numeric(data['x dimension'], errors='coerce')
data['y dimension'] = pd.to_numeric(data['y dimension'], errors='coerce')
data['z dimension'] = pd.to_numeric(data['z dimension'], errors='coerce')
data['depth'] = pd.to_numeric(data['depth'], errors='coerce')
data['table'] = pd.to_numeric(data['table'], errors='coerce')


data['clarity'] = data['clarity'].astype(str)
data['color'] = data['color'].astype(str)
data['cut'] = data['cut'].astype(str)

print(data.dtypes)

# change the spelling of cut, color and clarity to lower
# to ensure that the values represented as text are consistent
data['clarity'] = data['clarity'].str.lower()
data['color'] = data['color'].str.lower()
data['cut'] = data['cut'].str.lower()

#### Check null values and delete rows with null price

In [ ]:
# check the numebr of null values
print(data.isnull().sum())

# delete the rows with null price - price is our dependent variable vector
data = data.dropna(subset=['price'])

# filter out the data where prices are extreme
lower_bound, upper_bound = data['price'].quantile(0.05), data['price'].quantile(0.95)
data = data[(data['price'] >= lower_bound) & (data['price'] <= upper_bound)]


#### Create clean data file for the dashboard

In [ ]:
data.to_csv("clean_data.csv", index=False)

#### Fill the missing feature values

In [ ]:
# replace the null numeric values with the mean of the column values
# we do this only for the features
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

replace_cols = ['carat', 'x dimension', 'y dimension', 'z dimension', 'depth', 'table']
imputer.fit(data[replace_cols])
data[replace_cols] = imputer.transform(data[replace_cols]) # replace the missing values

print(f"{data.head(3)} \n")

## Regression model

#### One-hot encoding

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

categorical_vars = ['clarity', 'color', 'cut']

encoder = OneHotEncoder()
transformer = ColumnTransformer([("encoder", encoder, categorical_vars)], remainder='passthrough')

encoded_data = transformer.fit_transform(data)

encoded_columns = transformer.get_feature_names_out()[:-7] # do not take the remainders
remainder_columns = [col for col in data.columns if col not in categorical_vars]
new_columns = np.append(encoded_columns, remainder_columns)

# create new dataframe to maintain meaningful column names
encoded_data = pd.DataFrame(encoded_data, columns=new_columns)
print(encoded_data.head())

#### Create the feature matrix and dependent variable vector

In [ ]:
X = encoded_data.drop('price', axis=1)
y = encoded_data['price']

#### Backward elimination

In [ ]:
import statsmodels.api as sm


def backward_elimination(data, target, significance_level=0.05):
    features = data.columns.tolist()
    while len(features) > 0:
        features_with_constant = sm.add_constant(data[features])
        p_values = sm.OLS(target, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if max_p_value > significance_level:
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break
    return features

selected_features = backward_elimination(X, y)
print(selected_features)

#### Regression model

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from joblib import dump
from sklearn.metrics import mean_squared_error, r2_score

# divide the data into test and train
X_selected = X[selected_features]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)
dump(model, 'model.joblib')
dump(X_selected, 'input_data.joblib')

# measure the model accuracy on test data
y_pred = model.predict(X_test)
print('R^2:', r2_score(y_test, y_pred))

# show the models effect on a plot
plt.scatter(y_test, y_pred)
plt.xlabel('Rzeczywiste ceny')
plt.ylabel('Przewidywane ceny')
plt.title('Rzeczywiste vs Przewidywane ceny')
plt.plot([min(y_test), max(y_test)], [min(y_pred), max(y_pred)], color='red')
plt.show()